In [ ]:
import requests
import os

In [2]:
def load_genes_from_file(filepath):
    """Load gene IDs from a file, one per line."""
    with open(filepath, "r") as file:
        genes = [line.split("\t")[0] for line in file if line.strip()]
    return genes


In [ ]:

def run_panther_overrep(gene_list, organism="10090", annotation_type="GO:0008150"):
    """
    Perform PANTHER overrepresentation analysis via API.
    
    Parameters:
        gene_list (list): List of gene identifiers (e.g., Ensembl IDs).
        organism (str): NCBI Taxonomy ID (e.g., 9606 for human, 10090 for mouse).
        annotation_type (str): Type of GO annotation ('biological_process', 'molecular_function', etc.).
    
    Returns:
        dict: Parsed results of the enrichment analysis.
    """
    url = "https://pantherdb.org/services/oai/pantherdb/enrich/overrep"

    params = {
        "geneInputList": ",".join(gene_list),
        "organism": organism,
        "annotDataSet": annotation_type
    }
    response = requests.get(url, params=params)

    # Check response
    if response.status_code == 200:
        result = response.json()
        return result
    else:
        raise Exception(f"Error {response.status_code}: {response.text}")



In [ ]:
import csv

def save_results_to_tsv(results, output_file):
    """
    Save filtered enrichment results to a TSV file.
    """
    with open(output_file, mode='w', newline='', encoding='utf-8') as f:
        writer = csv.writer(f, delimiter='\t')
        
        # Header
        writer.writerow([
            "ID", "FDR", "Observed", "Reference", "Term"
        ])

        for result in results.get("results", {}).get("result", []):
            if result["fdr"] <= 0.05:
                term = result["term"]
                writer.writerow([
                    term.get("id"),
                    #f"{result['pValue']:.2e}",
                    f"{result['fdr']:.2e}",
                    #result.get("fold_enrichment"),
                    result.get("number_in_list"),
                    result.get("number_in_reference"),
                    #f"{result.get('expected', 0):.2f}",
                    f'\'{term.get("label")}\''
                ])


In [5]:
def save_mapped(response, output):
    mapped_ids  = response['results']['input_list'][ 'mapped_ids'].split(',')
# Write to file with one ID per line
    with open(output, 'w') as f:
        for gene_id in mapped_ids:
            if gene_id.strip():  # Skip empty entries
                f.write(f"{gene_id.strip()}\n")

In [ ]:

def routine_overrrep(input, output, annot):
    mouse_id=10090

    try:
        gene_list=load_genes_from_file(input)
        print("Loaded genes:", len(gene_list))
        results = run_panther_overrep(gene_list, organism=mouse_id, annotation_type=annot)
        save_results_to_tsv(results, output)
        print(f"Results saved to {output}")
        return results
        
    except Exception as e:
        print(f"Failed to run enrichment: {e}")


## View 1&2

In [ ]:
work_dir =  "./Ontology"
views = ["view1-padj" ,"view2-lfc2-p05"]
suffixes = ["OE-female", "OE-male", "VNO-female", "VNO-male"]

annot_BP="GO:0008150"
annot_MF="GO:0003674"

for view in views:
    for suff in suffixes:
        file = view + '_' + suff 
        in_file=os.path.join(work_dir, view, file + '.tsv')
        out_BP_file = os.path.join(work_dir, view, "PANTHER", file  + '.BP.out.tsv')
        out_MF_file = os.path.join(work_dir, view, "PANTHER", file  + '.MF.out.tsv')
        print("Processing:", os.path.basename(in_file), " for BP ...")
        response = routine_overrrep(in_file, out_BP_file, annot_BP)
        print("Processing:", os.path.basename(in_file), " for MF ...")
        routine_overrrep(in_file, out_MF_file, annot_MF)
        out_map = os.path.join(work_dir, view, "PANTHER", file  + '.map.out.tsv')
        save_mapped(response, out_map)

Processing: view1-padj_OE-female.tsv  for BP ...
genes: 3
Success!!! 1
Results saved to C:\Users\misog\Documents\CUNI\sem06\bc_data\Ontology2\view1-padj\PANTHER\view1-padj_OE-female.BP.out.tsv
Processing: view1-padj_OE-female.tsv  for MF ...
genes: 3
Success!!! 1
Results saved to C:\Users\misog\Documents\CUNI\sem06\bc_data\Ontology2\view1-padj\PANTHER\view1-padj_OE-female.MF.out.tsv
Processing: view1-padj_OE-male.tsv  for BP ...
genes: 6
Success!!! 1
Results saved to C:\Users\misog\Documents\CUNI\sem06\bc_data\Ontology2\view1-padj\PANTHER\view1-padj_OE-male.BP.out.tsv
Processing: view1-padj_OE-male.tsv  for MF ...
genes: 6
Failed to run enrichment: ('Connection broken: InvalidChunkLength(got length b\'list": 0,\\n\', 0 bytes read)', InvalidChunkLength(got length b'list": 0,\n', 0 bytes read))
Processing: view1-padj_VNO-female.tsv  for BP ...
genes: 25
Success!!! 1
Results saved to C:\Users\misog\Documents\CUNI\sem06\bc_data\Ontology2\view1-padj\PANTHER\view1-padj_VNO-female.BP.out.tsv


## View 3

In [ ]:
work_dir =  "./Ontology"
view = "view3-mean-h100"
suffixes = ["OE", "VNO"]

annot_BP="GO:0008150"
annot_MF="GO:0003674"


for suff in suffixes:
    file = view + '_' + suff
    in_file=os.path.join(work_dir, view, file + '.tsv')

    out_file = os.path.join(work_dir, view, "PANTHER", file  + '.BP.out.tsv')
    out_MF_file = os.path.join(work_dir, view, "PANTHER", file  + '.MF.out.tsv')

    print("Processing:", os.path.basename(in_file), " for BP ...")
    routine_overrrep(in_file, out_file, annot_BP)

    print("Processing:", os.path.basename(in_file), " for MF ...")
    response = routine_overrrep(in_file, out_MF_file, annot_MF)

    out_map = os.path.join(work_dir, view, "PANTHER", file  + '.map.out.tsv')
    save_mapped(response, out_map)

Processing: view3-mean-h100_OE.tsv  for BP ...
genes: 100
Success!!! 1
Results saved to C:\Users\misog\Documents\CUNI\sem06\bc_data\Ontology2\view3-mean-h100\PANTHER\view3-mean-h100_OE.BP.out.tsv
Processing: view3-mean-h100_OE.tsv  for MF ...
genes: 100
Success!!! 1
Results saved to C:\Users\misog\Documents\CUNI\sem06\bc_data\Ontology2\view3-mean-h100\PANTHER\view3-mean-h100_OE.MF.out.tsv
Processing: view3-mean-h100_VNO.tsv  for BP ...
genes: 100
Success!!! 1
Results saved to C:\Users\misog\Documents\CUNI\sem06\bc_data\Ontology2\view3-mean-h100\PANTHER\view3-mean-h100_VNO.BP.out.tsv
Processing: view3-mean-h100_VNO.tsv  for MF ...
genes: 100
Success!!! 1
Results saved to C:\Users\misog\Documents\CUNI\sem06\bc_data\Ontology2\view3-mean-h100\PANTHER\view3-mean-h100_VNO.MF.out.tsv
